In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras as keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import utils

Using TensorFlow backend.
C:\Users\Coss\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Coss\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Coss\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Coss\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

## Загрузка набора данных

In [2]:
data = pd.read_csv('11.txt', sep='\t', header=0, quotechar='"')
data = data.dropna()


data.head()

,Cr,Ni,Mo,Mn,Si,Nb,Ti,Zr,Ta,V,...,Grains mm-2,Type of melting,Size of ingot,Product form,Temperature (K),0.2%proof_stress (M Pa),UTS (M Pa),Elongation (%),Area_reduction (%),Comments
66,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,298,210.0,620.0,71.0,81.0,NRIM/CDS/No4B/1986 ABL
67,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,373,207.0,503.0,62.0,82.0,NRIM/CDS/No4B/1986 ABL
68,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,473,177.0,465.0,46.0,77.0,NRIM/CDS/No4B/1986 ABL
69,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,573,162.0,468.0,45.0,71.0,NRIM/CDS/No4B/1986 ABL
70,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,673,154.0,463.0,47.0,73.0,NRIM/CDS/No4B/1986 ABL


In [3]:
# y - то что будем предсказывать
# X - данный, по которым предсказывается y

y = data['UTS (M Pa)']

X = data.drop('UTS (M Pa)', axis=1)

# убираем лишнее

X = X.drop(['Comments', '0.2%proof_stress (M Pa)', 'Grains mm-2', 'Type of melting', 
           'Elongation (%)', 'Area_reduction (%)'], axis=1)



X.head()





,Cr,Ni,Mo,Mn,Si,Nb,Ti,Zr,Ta,V,...,Al,Sn,Pb,Solution_treatment_temperature,Solution_treatment_time(s),Water_Quenched_after_s.t.,Air_Quenched_after_s.t.,Size of ingot,Product form,Temperature (K)
66,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0.015,0,0,1343.0,600.0,1.0,0.0,4.5,1,298
67,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0.015,0,0,1343.0,600.0,1.0,0.0,4.5,1,373
68,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0.015,0,0,1343.0,600.0,1.0,0.0,4.5,1,473
69,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0.015,0,0,1343.0,600.0,1.0,0.0,4.5,1,573
70,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0.015,0,0,1343.0,600.0,1.0,0.0,4.5,1,673


## Разделение датасета на тренировочную и тестовую части

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    # доля объёма тестового множества
                                                    test_size=0.3)



print(X_train)


        Cr     Ni     Mo    Mn    Si     Nb     Ti  Zr  Ta      V  ...     Al  \
572  17.26  12.21  0.070  1.70  0.80  0.770  0.018   0   0  0.032  ...  0.012   
406  21.00  32.34  0.002  1.12  0.43  0.000  0.490   0   0  0.000  ...  0.250   
569  17.26  12.21  0.070  1.70  0.80  0.770  0.018   0   0  0.032  ...  0.012   
508  20.30  31.86  0.040  1.06  0.53  0.000  0.490   0   0  0.000  ...  0.520   
303  17.31  13.04  2.560  1.70  0.74  0.040  0.011   0   0  0.000  ...  0.005   
..     ...    ...    ...   ...   ...    ...    ...  ..  ..    ...  ...    ...   
215  16.42  13.21  2.340  1.51  0.52  0.010  0.011   0   0  0.000  ...  0.018   
178  17.71  12.62  0.020  1.67  0.54  0.005  0.530   0   0  0.000  ...  0.161   
491  20.19  32.37  0.090  0.92  0.46  0.000  0.450   0   0  0.000  ...  0.440   
351  16.97  10.28  2.440  1.75  0.81  0.040  0.044   0   0  0.000  ...  0.040   
365  17.30  10.42  2.370  1.67  0.58  0.030  0.047   0   0  0.000  ...  0.030   

     Sn  Pb  Solution_treat

## Преобразовывание данных в матричный вид

In [5]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

print(X_train)


[[1.726e+01 1.221e+01 7.000e-02 ... 6.000e+00 1.000e+00 8.230e+02]
 [2.100e+01 3.234e+01 2.000e-03 ... 9.000e-01 1.000e+00 5.730e+02]
 [1.726e+01 1.221e+01 7.000e-02 ... 6.000e+00 1.000e+00 5.730e+02]
 ...
 [2.019e+01 3.237e+01 9.000e-02 ... 1.600e+00 2.000e+00 2.980e+02]
 [1.697e+01 1.028e+01 2.440e+00 ... 3.500e-01 3.000e+00 7.730e+02]
 [1.730e+01 1.042e+01 2.370e+00 ... 3.500e-01 3.000e+00 1.023e+03]]


## Нормализация данных

In [6]:

from sklearn.preprocessing import MinMaxScaler 
scale_features_mm = MinMaxScaler() 
X_train = scale_features_mm.fit_transform(X_train) 
X_test = scale_features_mm.transform(X_test) 

print(X_train)


[[0.1862955  0.12612967 0.02658327 ... 0.36668778 0.         0.53846154]
 [0.98715203 0.91709234 0.         ... 0.04369854 0.         0.28205128]
 [0.1862955  0.12612967 0.02658327 ... 0.36668778 0.         0.28205128]
 ...
 [0.8137045  0.91827112 0.03440188 ... 0.0880304  0.5        0.        ]
 [0.124197   0.0502947  0.95308835 ... 0.00886637 1.         0.48717949]
 [0.19486081 0.05579568 0.92572322 ... 0.00886637 1.         0.74358974]]


In [7]:
from keras import regularizers
from keras.layers import Dropout

model = Sequential()
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.001),  input_shape=(X_train.shape[1],)))
model.add(Dense(2048, activation='relu', ))
model.add(Dense(1024, activation='relu', ))
model.add(Dense(2048, activation='relu', ))


model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

## Обучение нейронной сети

In [8]:
model.fit(X_train, y_train, epochs=1500, batch_size=64, verbose=2)


Epoch 1/1500
 - 0s - loss: 175475.1480 - mae: 397.7021
Epoch 2/1500
 - 0s - loss: 109350.5922 - mae: 300.5149
Epoch 3/1500
 - 0s - loss: 48709.2518 - mae: 172.2568
Epoch 4/1500
 - 0s - loss: 30397.4314 - mae: 148.0676
Epoch 5/1500
 - 0s - loss: 36246.0571 - mae: 172.5902
Epoch 6/1500
 - 0s - loss: 22654.1949 - mae: 111.9930
Epoch 7/1500
 - 0s - loss: 20749.7903 - mae: 106.3147
Epoch 8/1500
 - 0s - loss: 19043.3481 - mae: 121.1813
Epoch 9/1500
 - 0s - loss: 14525.1922 - mae: 96.5643
Epoch 10/1500
 - 0s - loss: 13464.9269 - mae: 82.9485
Epoch 11/1500
 - 0s - loss: 11288.0590 - mae: 89.0496
Epoch 12/1500
 - 0s - loss: 9280.8441 - mae: 75.6586
Epoch 13/1500
 - 0s - loss: 8038.9406 - mae: 66.6003
Epoch 14/1500
 - 0s - loss: 6847.2552 - mae: 67.3080
Epoch 15/1500
 - 0s - loss: 5737.7260 - mae: 58.7138
Epoch 16/1500
 - 0s - loss: 4967.0702 - mae: 56.7010
Epoch 17/1500
 - 0s - loss: 4385.4325 - mae: 55.1619
Epoch 18/1500
 - 0s - loss: 4472.7785 - mae: 55.5191
Epoch 19/1500
 - 0s - loss: 4333.

Epoch 152/1500
 - 0s - loss: 383.0123 - mae: 14.9763
Epoch 153/1500
 - 0s - loss: 320.4600 - mae: 13.8108
Epoch 154/1500
 - 0s - loss: 348.4171 - mae: 14.2774
Epoch 155/1500
 - 0s - loss: 311.2562 - mae: 12.8688
Epoch 156/1500
 - 0s - loss: 314.9954 - mae: 13.5251
Epoch 157/1500
 - 0s - loss: 338.6239 - mae: 13.8027
Epoch 158/1500
 - 0s - loss: 368.1488 - mae: 14.7868
Epoch 159/1500
 - 0s - loss: 389.7064 - mae: 15.1681
Epoch 160/1500
 - 0s - loss: 365.6834 - mae: 15.1426
Epoch 161/1500
 - 0s - loss: 377.5683 - mae: 14.8885
Epoch 162/1500
 - 0s - loss: 309.7400 - mae: 13.3230
Epoch 163/1500
 - 0s - loss: 277.6468 - mae: 12.6271
Epoch 164/1500
 - 0s - loss: 292.4666 - mae: 12.1077
Epoch 165/1500
 - 0s - loss: 252.4235 - mae: 11.2199
Epoch 166/1500
 - 0s - loss: 270.3209 - mae: 11.7265
Epoch 167/1500
 - 0s - loss: 297.1055 - mae: 12.8661
Epoch 168/1500
 - 0s - loss: 267.4689 - mae: 12.0817
Epoch 169/1500
 - 0s - loss: 253.5972 - mae: 11.4709
Epoch 170/1500
 - 0s - loss: 247.4434 - mae: 1

Epoch 308/1500
 - 0s - loss: 116.4868 - mae: 8.2480
Epoch 309/1500
 - 0s - loss: 144.1734 - mae: 9.1735
Epoch 310/1500
 - 0s - loss: 132.2780 - mae: 8.7276
Epoch 311/1500
 - 0s - loss: 136.9957 - mae: 9.1281
Epoch 312/1500
 - 0s - loss: 149.3509 - mae: 9.6133
Epoch 313/1500
 - 0s - loss: 130.8302 - mae: 8.6502
Epoch 314/1500
 - 0s - loss: 85.5807 - mae: 6.5621
Epoch 315/1500
 - 0s - loss: 61.1410 - mae: 5.3897
Epoch 316/1500
 - 0s - loss: 61.8521 - mae: 5.3657
Epoch 317/1500
 - 0s - loss: 67.7057 - mae: 5.5811
Epoch 318/1500
 - 0s - loss: 53.4902 - mae: 5.0307
Epoch 319/1500
 - 0s - loss: 53.2875 - mae: 4.8723
Epoch 320/1500
 - 0s - loss: 53.8925 - mae: 4.8462
Epoch 321/1500
 - 0s - loss: 48.8475 - mae: 4.6288
Epoch 322/1500
 - 0s - loss: 58.5174 - mae: 5.4165
Epoch 323/1500
 - 0s - loss: 48.9081 - mae: 4.8816
Epoch 324/1500
 - 0s - loss: 53.8517 - mae: 5.1832
Epoch 325/1500
 - 0s - loss: 65.7636 - mae: 6.0185
Epoch 326/1500
 - 0s - loss: 72.3628 - mae: 6.1705
Epoch 327/1500
 - 0s - lo

Epoch 469/1500
 - 0s - loss: 37.3075 - mae: 4.6359
Epoch 470/1500
 - 0s - loss: 82.2449 - mae: 7.4837
Epoch 471/1500
 - 0s - loss: 95.5978 - mae: 8.0458
Epoch 472/1500
 - 0s - loss: 68.8766 - mae: 6.4111
Epoch 473/1500
 - 0s - loss: 55.8809 - mae: 5.8500
Epoch 474/1500
 - 0s - loss: 66.7121 - mae: 6.4585
Epoch 475/1500
 - 0s - loss: 82.0543 - mae: 7.0643
Epoch 476/1500
 - 0s - loss: 128.1103 - mae: 9.1321
Epoch 477/1500
 - 0s - loss: 83.4550 - mae: 7.4717
Epoch 478/1500
 - 0s - loss: 63.2412 - mae: 6.1403
Epoch 479/1500
 - 0s - loss: 52.7821 - mae: 5.7771
Epoch 480/1500
 - 0s - loss: 48.8130 - mae: 5.3245
Epoch 481/1500
 - 0s - loss: 35.1017 - mae: 4.1998
Epoch 482/1500
 - 0s - loss: 24.4748 - mae: 3.5649
Epoch 483/1500
 - 0s - loss: 22.5750 - mae: 3.5669
Epoch 484/1500
 - 0s - loss: 22.0467 - mae: 3.5499
Epoch 485/1500
 - 0s - loss: 21.0347 - mae: 3.4038
Epoch 486/1500
 - 0s - loss: 30.4086 - mae: 4.2347
Epoch 487/1500
 - 0s - loss: 34.3667 - mae: 4.6540
Epoch 488/1500
 - 0s - loss: 3

Epoch 629/1500
 - 0s - loss: 24.6083 - mae: 3.9199
Epoch 630/1500
 - 0s - loss: 21.2512 - mae: 3.5803
Epoch 631/1500
 - 0s - loss: 27.1572 - mae: 4.1370
Epoch 632/1500
 - 0s - loss: 24.7454 - mae: 3.8816
Epoch 633/1500
 - 0s - loss: 39.9775 - mae: 5.2769
Epoch 634/1500
 - 0s - loss: 26.2104 - mae: 3.9324
Epoch 635/1500
 - 0s - loss: 23.0443 - mae: 3.6546
Epoch 636/1500
 - 0s - loss: 21.1268 - mae: 3.5583
Epoch 637/1500
 - 0s - loss: 37.1789 - mae: 4.6989
Epoch 638/1500
 - 0s - loss: 59.4585 - mae: 6.4214
Epoch 639/1500
 - 0s - loss: 39.3795 - mae: 5.0670
Epoch 640/1500
 - 0s - loss: 39.3008 - mae: 5.0264
Epoch 641/1500
 - 0s - loss: 33.9436 - mae: 4.7738
Epoch 642/1500
 - 0s - loss: 23.3103 - mae: 3.7655
Epoch 643/1500
 - 0s - loss: 19.9111 - mae: 3.3496
Epoch 644/1500
 - 0s - loss: 20.2557 - mae: 3.5844
Epoch 645/1500
 - 0s - loss: 18.6253 - mae: 3.3824
Epoch 646/1500
 - 0s - loss: 26.2777 - mae: 4.0614
Epoch 647/1500
 - 0s - loss: 22.2837 - mae: 3.6435
Epoch 648/1500
 - 0s - loss: 20

Epoch 790/1500
 - 0s - loss: 95.9296 - mae: 8.0161
Epoch 791/1500
 - 0s - loss: 83.6064 - mae: 7.1725
Epoch 792/1500
 - 0s - loss: 227.9597 - mae: 12.9666
Epoch 793/1500
 - 0s - loss: 286.1375 - mae: 14.3812
Epoch 794/1500
 - 0s - loss: 145.0767 - mae: 9.5409
Epoch 795/1500
 - 0s - loss: 138.4180 - mae: 9.5126
Epoch 796/1500
 - 0s - loss: 137.6979 - mae: 9.4678
Epoch 797/1500
 - 0s - loss: 107.4728 - mae: 7.5453
Epoch 798/1500
 - 0s - loss: 200.8864 - mae: 11.4585
Epoch 799/1500
 - 0s - loss: 152.6762 - mae: 9.6898
Epoch 800/1500
 - 0s - loss: 152.6065 - mae: 10.1904
Epoch 801/1500
 - 0s - loss: 129.5419 - mae: 9.2004
Epoch 802/1500
 - 0s - loss: 224.5963 - mae: 12.6080
Epoch 803/1500
 - 0s - loss: 211.8869 - mae: 12.5394
Epoch 804/1500
 - 0s - loss: 162.9187 - mae: 10.6458
Epoch 805/1500
 - 0s - loss: 174.6015 - mae: 10.6417
Epoch 806/1500
 - 0s - loss: 229.3660 - mae: 12.7124
Epoch 807/1500
 - 0s - loss: 110.0488 - mae: 7.9346
Epoch 808/1500
 - 0s - loss: 89.4995 - mae: 7.1260
Epoch 

Epoch 951/1500
 - 0s - loss: 34.6643 - mae: 4.6193
Epoch 952/1500
 - 0s - loss: 23.7867 - mae: 3.7903
Epoch 953/1500
 - 0s - loss: 69.9917 - mae: 6.8552
Epoch 954/1500
 - 0s - loss: 40.8047 - mae: 5.2015
Epoch 955/1500
 - 0s - loss: 30.5365 - mae: 4.3976
Epoch 956/1500
 - 0s - loss: 57.0960 - mae: 6.3238
Epoch 957/1500
 - 0s - loss: 86.3175 - mae: 7.7190
Epoch 958/1500
 - 0s - loss: 256.2267 - mae: 13.4327
Epoch 959/1500
 - 0s - loss: 266.7741 - mae: 13.6136
Epoch 960/1500
 - 0s - loss: 512.1412 - mae: 18.4325
Epoch 961/1500
 - 0s - loss: 607.1904 - mae: 20.8032
Epoch 962/1500
 - 0s - loss: 556.9568 - mae: 19.3804
Epoch 963/1500
 - 0s - loss: 860.1816 - mae: 23.8001
Epoch 964/1500
 - 0s - loss: 1120.6261 - mae: 28.6278
Epoch 965/1500
 - 0s - loss: 943.0248 - mae: 24.8683
Epoch 966/1500
 - 0s - loss: 553.5328 - mae: 19.9073
Epoch 967/1500
 - 0s - loss: 371.6453 - mae: 14.7204
Epoch 968/1500
 - 0s - loss: 322.6832 - mae: 14.3743
Epoch 969/1500
 - 0s - loss: 164.3370 - mae: 10.3412
Epoch 

Epoch 1110/1500
 - 0s - loss: 10.1571 - mae: 2.3506
Epoch 1111/1500
 - 0s - loss: 8.7120 - mae: 2.1957
Epoch 1112/1500
 - 0s - loss: 10.5962 - mae: 2.4570
Epoch 1113/1500
 - 0s - loss: 11.5046 - mae: 2.5509
Epoch 1114/1500
 - 0s - loss: 14.0244 - mae: 2.9760
Epoch 1115/1500
 - 0s - loss: 7.8377 - mae: 1.9624
Epoch 1116/1500
 - 0s - loss: 11.5278 - mae: 2.6724
Epoch 1117/1500
 - 0s - loss: 13.7687 - mae: 2.8389
Epoch 1118/1500
 - 0s - loss: 10.6070 - mae: 2.4497
Epoch 1119/1500
 - 0s - loss: 8.9629 - mae: 2.1675
Epoch 1120/1500
 - 0s - loss: 9.5810 - mae: 2.2291
Epoch 1121/1500
 - 0s - loss: 18.7567 - mae: 3.3777
Epoch 1122/1500
 - 0s - loss: 13.5234 - mae: 2.7867
Epoch 1123/1500
 - 0s - loss: 9.3923 - mae: 2.2580
Epoch 1124/1500
 - 0s - loss: 8.8016 - mae: 2.1541
Epoch 1125/1500
 - 0s - loss: 7.3445 - mae: 1.9637
Epoch 1126/1500
 - 0s - loss: 10.3311 - mae: 2.5015
Epoch 1127/1500
 - 0s - loss: 14.9883 - mae: 3.1614
Epoch 1128/1500
 - 0s - loss: 34.2535 - mae: 4.7277
Epoch 1129/1500
 - 

Epoch 1268/1500
 - 0s - loss: 26.0023 - mae: 3.8554
Epoch 1269/1500
 - 0s - loss: 18.2822 - mae: 3.2190
Epoch 1270/1500
 - 0s - loss: 16.5315 - mae: 3.0495
Epoch 1271/1500
 - 0s - loss: 14.4137 - mae: 2.9138
Epoch 1272/1500
 - 0s - loss: 26.7360 - mae: 3.9604
Epoch 1273/1500
 - 0s - loss: 15.1285 - mae: 2.9059
Epoch 1274/1500
 - 0s - loss: 21.4020 - mae: 3.7806
Epoch 1275/1500
 - 0s - loss: 16.8370 - mae: 3.1179
Epoch 1276/1500
 - 0s - loss: 20.3148 - mae: 3.6223
Epoch 1277/1500
 - 0s - loss: 10.1892 - mae: 2.3625
Epoch 1278/1500
 - 0s - loss: 13.7680 - mae: 2.8681
Epoch 1279/1500
 - 0s - loss: 10.4985 - mae: 2.3719
Epoch 1280/1500
 - 0s - loss: 8.4288 - mae: 2.0213
Epoch 1281/1500
 - 0s - loss: 7.2626 - mae: 1.9065
Epoch 1282/1500
 - 0s - loss: 6.3995 - mae: 1.7900
Epoch 1283/1500
 - 0s - loss: 7.5413 - mae: 1.9144
Epoch 1284/1500
 - 0s - loss: 4.5385 - mae: 1.3983
Epoch 1285/1500
 - 0s - loss: 5.3367 - mae: 1.6095
Epoch 1286/1500
 - 0s - loss: 3.9814 - mae: 1.2150
Epoch 1287/1500
 - 

Epoch 1426/1500
 - 0s - loss: 72.7080 - mae: 6.9873
Epoch 1427/1500
 - 0s - loss: 85.6583 - mae: 7.5248
Epoch 1428/1500
 - 0s - loss: 136.6101 - mae: 9.6354
Epoch 1429/1500
 - 0s - loss: 159.1364 - mae: 10.4330
Epoch 1430/1500
 - 0s - loss: 213.7931 - mae: 12.4400
Epoch 1431/1500
 - 0s - loss: 310.8803 - mae: 15.0631
Epoch 1432/1500
 - 0s - loss: 196.9683 - mae: 11.9173
Epoch 1433/1500
 - 0s - loss: 197.4099 - mae: 11.7779
Epoch 1434/1500
 - 0s - loss: 283.6899 - mae: 13.5622
Epoch 1435/1500
 - 0s - loss: 294.7283 - mae: 14.1307
Epoch 1436/1500
 - 0s - loss: 184.1017 - mae: 10.4472
Epoch 1437/1500
 - 0s - loss: 74.8450 - mae: 6.7326
Epoch 1438/1500
 - 0s - loss: 74.8635 - mae: 6.8131
Epoch 1439/1500
 - 0s - loss: 100.1669 - mae: 8.5439
Epoch 1440/1500
 - 0s - loss: 47.9469 - mae: 5.2954
Epoch 1441/1500
 - 0s - loss: 69.5338 - mae: 6.5416
Epoch 1442/1500
 - 0s - loss: 58.5171 - mae: 5.6944
Epoch 1443/1500
 - 0s - loss: 69.1774 - mae: 7.0227
Epoch 1444/1500
 - 0s - loss: 52.0158 - mae: 5

## Оценка точности нейронной сети

In [9]:
mse, mae = model.evaluate(X_test, y_test, verbose=0)

In [10]:
pred = model.predict(X_test)

In [11]:
print("Средняя абсолютная ошибка:", mae)

Средняя абсолютная ошибка: 11.548542976379395


## Проверка на тестовых данных

In [12]:
print("Предсказанное значение:", pred[1][0], ", правильное значение:", y_test[1])
print("Предсказанное значение:", pred[2][0], ", правильное значение:", y_test[2])
print("Предсказанное значение:", pred[3][0], ", правильное значение:", y_test[3])
print("Предсказанное значение:", pred[4][0], ", правильное значение:", y_test[4])
print("Предсказанное значение:", pred[10][0], ", правильное значение:", y_test[10])
print("Предсказанное значение:", pred[12][0], ", правильное значение:", y_test[12])
print("Предсказанное значение:", pred[14][0], ", правильное значение:", y_test[14])
print("Предсказанное значение:", pred[16][0], ", правильное значение:", y_test[16])
print("Предсказанное значение:", pred[18][0], ", правильное значение:", y_test[18])
print("Предсказанное значение:", pred[30][0], ", правильное значение:", y_test[30])
print("Предсказанное значение:", pred[35][0], ", правильное значение:", y_test[35])
print("Предсказанное значение:", pred[40][0], ", правильное значение:", y_test[40])
print("Предсказанное значение:", pred[45][0], ", правильное значение:", y_test[45])
print("Предсказанное значение:", pred[50][0], ", правильное значение:", y_test[50])
print("Предсказанное значение:", pred[60][0], ", правильное значение:", y_test[60])

Предсказанное значение: 310.47833 , правильное значение: 313.0
Предсказанное значение: 529.239 , правильное значение: 560.0
Предсказанное значение: 304.90573 , правильное значение: 318.0
Предсказанное значение: 359.79245 , правильное значение: 374.0
Предсказанное значение: 292.11124 , правильное значение: 293.0
Предсказанное значение: 290.03925 , правильное значение: 300.0
Предсказанное значение: 456.77817 , правильное значение: 454.0
Предсказанное значение: 145.9669 , правильное значение: 151.0
Предсказанное значение: 586.6688 , правильное значение: 580.0
Предсказанное значение: 386.01572 , правильное значение: 396.0
Предсказанное значение: 369.2487 , правильное значение: 378.0
Предсказанное значение: 371.25882 , правильное значение: 370.0
Предсказанное значение: 427.97464 , правильное значение: 412.0
Предсказанное значение: 451.2029 , правильное значение: 458.0
Предсказанное значение: 188.8001 , правильное значение: 188.0
